In [1]:
import pandas as pd

In [17]:
%cd ..
%cd Coursera/Coursera/CSVs/

C:\Users\AMOS OSHIYOYE\Documents\VSCODE_user_files
C:\Users\AMOS OSHIYOYE\Documents\VSCODE_user_files\Coursera\Coursera\CSVs


In [18]:
df=pd.read_csv('ChicagoPublicSchools.csv')

In [296]:
for a in df.drop('COMMUNITY_AREA_NAME',axis=1).columns:
    if df[a].dtypes=='object' and df[a].str.contains('NDA').any() and (df[a].value_counts()['NDA']>(df.shape[0]/2)):
        df1=df.drop([a],axis=1)

In [19]:
df['AVERAGE_STUDENT_ATTENDANCE']=df['AVERAGE_STUDENT_ATTENDANCE'].apply(lambda x: float(str(x).strip('%')))

In [20]:
zip_sch_type=df.groupby('ZIP_Code')['Elementary, Middle, or High School'].value_counts(sort=False)
zip_safety=df.groupby('ZIP_Code')['SAFETY_SCORE'].mean()
zip_avg_att=df.groupby('ZIP_Code')['AVERAGE_STUDENT_ATTENDANCE'].mean()
zip_center=df.groupby('ZIP_Code')['Latitude','Longitude'].mean().reset_index()
chicago_midpoint=df.loc[:,['Latitude','Longitude']].mean()
# zip_center

C:\Users\AMOS OSHIYOYE\AppData\Local\Temp\ipykernel_808\2522552395.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  zip_center=df.groupby('ZIP_Code')['Latitude','Longitude'].mean().reset_index()


In [21]:
import folium
from folium import DivIcon,plugins

In [401]:
# i=6
df.at[i,'NAME_OF_SCHOOL']
# str.find(df.at[i,'NAME_OF_SCHOOL'],'Lang')

In [22]:
# lap=df.groupby(['ZIP_Code'])['Latitude','Longitude'].transform('median')
# lap.rename(columns={'Latitude':'Latitude_','Longitude':'Longitude_'}
# df['Latitude_']=lap.iloc[:,0]
df[['Latitude_','Longitude_']]=df.groupby(['ZIP_Code'])['Latitude','Longitude'].transform('median')

C:\Users\AMOS OSHIYOYE\AppData\Local\Temp\ipykernel_808\1374897365.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df[['Latitude_','Longitude_']]=df.groupby(['ZIP_Code'])['Latitude','Longitude'].transform('median')


%cd ..
%cd ..
%cd ..
%cd "Chicago Schools"

In [24]:
sch_map=folium.Map(location=chicago_midpoint,zoom_start=10,tiles='OpenStreetMap')
clusters=plugins.MarkerCluster().add_to(sch_map)
for i, rows in df.iterrows():
    lat=df.at[i,'Latitude_']
    long=df.at[i,'Longitude_']
    folium.Circle(location=[lat,long],radius=200,color='purple',).add_to(sch_map)
for i,row in df.iterrows():
    lat=df.at[i,'Latitude']
    long=df.at[i,'Longitude']
    name=df.at[i,'NAME_OF_SCHOOL']
    type_=df.at[i,'Elementary, Middle, or High School']
    web=df.at[i,'Link']
   
# str.find(df.at[i,'NAME_OF_SCHOOL'],'Lang')
    if type_=='ES':
        type_='Elementary Schools'
        color_='darkblue'
    elif type_=='MS':
        color_='darkred'
        type_='Middle School'
    elif type=='HS':
        type_='High School'
        color_='darkgreen'
    else:
        type_='Others'
    def sch_special():
        if str.find(df.at[i,'NAME_OF_SCHOOL'],'Lang')!=-1:
            icon_='globe'
        else:
            icon_='book'
        return icon_
            
    pop_up= "<div style='{background-color:grey,color=white}'>"+name+'<br> School Type: '+type_ + '<br> ' + str(web) + '<br></div>'
    folium.Marker(location=[lat,long],tooltip=name,popup=pop_up,icon=folium.Icon(icon=sch_special(),icon_color=color_,prefix='glyphicon',icon_size=(2,2),icon_anchor=(0,0))).add_to(clusters)
    # 
folium.Circle(location=chicago_midpoint, radius=400,tooltip="Proposed Site for the Chicago Central Library", color='black',fill=True,fill_color='lightblue').add_to(sch_map)
# folium.Marker(location=chicago_midpoint,,icon=folium.Icon(icon_size=(5,5))).add_to(sch_map)
sch_map.save('index.html')
sch_map

In [26]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [27]:
for a,b in df.iterrows():
    lat_z=df.at[a,'Latitude_']
    long_z=df.at[a,'Longitude_']
    lat=df.at[a,'Latitude']
    long=df.at[a,'Longitude']
    distance_coastline=calculate_distance(lat,long,lat_z,long_z)
    folium.Marker([lat,long],
    icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
    ).add_to(sch_map)
    folium.Polygon([[lat_z,long_z],[lat,long]]).add_to(sch_map)

sch_map.save('index.html')
# df['Latitude_']

In [438]:
df['Location'].head()

0    (41.92449696, -87.64452163)
1    (41.76032435, -87.55673627)
2    (41.74711093, -87.73170248)
3      (41.8097569, -87.6721446)
4    (41.82814609, -87.63279369)
Name: Location, dtype: object